# Adjoint Method & Backpropagation
<div style="text-align: right"> (C) Nikolai Nowaczyk, Jörg Kienitz 2020</div>

The adjoint method is a technique to speed up the calculation of derivatives for complicated functions, which can we written as a decomposition of simpler functions. It is widely used in applied mathematics such as oceanography and mathematical finance. We recap the method and show how it is related to the [backpropagation algorithm of neural networks](https://nbviewer.jupyter.org/github/niknow/machine-learning-examples/blob/master/newton_gradient_backprop/backpropagation.ipynb). 

# Derivatives and the Chain Rule

The key ingredient to the adjoint method is the chain rule.

**Theorem (chain rule):** Let $f_1:\mathbb{R}^{n_0} \to \mathbb{R}^{n_1}$ and $f_2:\mathbb{R}^{n_1} \to \mathbb{R}^{n_2}$ be two differentiable funcions and $f := f_2 \circ f_1: \mathbb{R}^{n_0} \to \mathbb{R}^{n_2}$ be the composition. Then $f$ is differentiable and
\begin{align}
    \forall x \in \mathbb{R}^{n_0}: \nabla f(x) = \nabla f_2 (f_1(x)) \bullet \nabla f_1(x),
\end{align}
where $\bullet$ denotes the ordinary matrix product. 

Remark: This theorem holds analogously on functions, which are only defined on open subsets.

# Forward method

Of course, the chain rule can be applied successively: Assume we are given a scalar function, which is a result of a complicated compisition of functions, i.e. we are given functions $f_l:\mathbb{R}^{n_{l-1}} \to \mathbb{R}^{n_l}$ and $g:\mathbb{R}^{n_L} \to \mathbb{R}$ and we want to calculate the derivative of the composition
\begin{align}
    F := g \circ f_L \circ \ldots \circ f_1: \mathbb{R}^{n_0} \to \mathbb{R}
\end{align}
at some point $x_0 \in \mathbb{R}^{n_0}$. Then the straight-forward way to calculate this would be to proceed in two steps:

1. Starting from $x_0$ successively compute
\begin{align}
    x_l := f_l(x_{l-1})
\end{align}
for $l=1, \ldots, L$ and finally  $y := g(x_L)$. 

2. Compute all the derivatives $\nabla f_l (x_{l-1})$ for $l=1, \ldots, L$ and $\nabla g(x_L)$ and then starting from $D_1 := \nabla f_1 (x_0)$ sucessively compute
\begin{align}
    D_l := \nabla f_l(x_{l}) \bullet D_{l-1}
\end{align}
and finally $D := \nabla g (x_L) \bullet D_L$.

Because of the chain rule
\begin{align}
    \nabla F(x_0) = \nabla g(x_L) \bullet \nabla f_L(x_{l-1}) \bullet \ldots \bullet \nabla f_1(x_0)
\end{align}
this *forward method* correctly computes $\nabla F(x_0)$.

## Complexity
The foward method computes $\nabla F(x_0)$ by a sequence of matrix multiplications starting on the right of the above equation. That means that in each step when $D_l$ is calculated from $D_{l-1}$ a matrix multiplication needs to be executed, which in a naive implementation has complexity $\mathcal{O}(n^3)$. 

**Key insight:** Because the last function $g$ in the chain is scalar valued, the derivative $\nabla g(x_L)$ is a vector and thus $\nabla F(x_0) = \nabla g(x_L) \bullet D_L$ is also a vector. Thus, computing the matrix products by the foward method just to arrive at a vector is not very efficient.

# Adjoint Method
The adjoint method computes the $\nabla F(x_0)$ using the exact same equation of the chain rule but not evaluated from right to left, but from left to right. Mathematically, one often formulates that by computing the adjoint equation instead, which in $\mathbb{R}^n$ simply amounts to taking the transpose:

\begin{align}
    \nabla F(x_0)^{\top} = \nabla f_1(x_0)^{\top} \bullet \ldots \bullet \nabla f_L(x_{L-1})^{\top} \bullet \nabla g(x_L)^{\top} 
\end{align}

Algorithmically, this means that the adjoint method starts by computing $V_{L+1} := \nabla g(x_L)^{\top}$ and then computes

\begin{align*}
    V_l := \nabla f_l(x_{l-1})^{\top} \bullet V_{l+1}
\end{align*}

backwards from $l=L, \ldots, 1$. 

This is a sequence of matrix-vector multiplications, which only as complexity $\mathcal{O}(n^2)$ and is thus much faster.

# Adjoint Method and Backpropagation

The forward method and the adjoint method relate to neural networks as follows:

Let $\operatorname{NN} = (A_l, b_l, \sigma_l)_{1 \leq l \leq L}$ be a neural network with $L$ layers. Its feed-forward function can be written as a composition $F = F_L \circ \ldots \circ F_1$ and for each $l$, we have $F_l = \sigma_l \circ f_{A_l, b_l}$. Here, for any matrix $A$ and $b$ of compatible dimensions, we denote by $f_{A,b}$ the function $v \mapsto Av + b$. 

For any fixed input $x=:a_0$, we can compute the feed-forward via
\begin{align}
    z_l := f_{A_l, b_l}(a_{l-1}), && a_l := \sigma_l(z_l)
\end{align}
for $l=1, \ldots, L$. This is exactly the first step of the forward method above.

Now, for that fixed input $x$ also fix a cost function $C$. Then the function $C \circ F$ is exactly of the shape above, i.e. it is a composition
\begin{align}
    C\circ F &= (C \circ \sigma_L) \circ (f_{A_L, b_L} \circ \sigma_{L-1}) \circ \ldots \sigma_2 \circ (f_{A_2, b_2} \circ \sigma_1) \circ f_{A_1, b_1} \\
    &=: g \circ h_L \circ \ldots h_2 \circ f_{A_1, b_1},
\end{align}
where $g := C \circ \sigma_L$ and $h_l := f_{A_{l+1}, b_{l+1}} \circ \sigma_{l}$. Thus, its transposed derivative is given via
\begin{align}
    \nabla (C\circ F)(x)^{\top} = \nabla f_{A_1, b_1}(a_0)^{\top} \bullet \nabla h_2(z_2)^{\top} \ldots \bullet \nabla h_{L-1}(z_{L-1})^{\top} \bullet \nabla g(z_L)^{\top}
\end{align}
and can be computed via the adjoint method by a backwards sequence of vectors $V_l$, where
\begin{align}
    V_{L} & = \nabla g(z_L)^{\top} = \nabla \sigma_L(z_L)^{\top} \bullet \nabla C(a_L)^{\top}
\end{align}
and for all $l=L-1, \ldots, 2$
\begin{align}
    V_{l} = \nabla h_l(z_l)^{\top} \bullet V_{l+1} 
    = \nabla(f_{A_{l+1},b_{l+1}} \circ \sigma_{l})(z_l)^{\top} \bullet V_{l+1} 
    = \nabla \sigma_{l}(z_l)^{\top} \bullet A_{l+1}^{\top} \bullet V_{l+1}
\end{align}
and finally
\begin{align}
    V_1 = \nabla f_{A_1, b_1}(a_0)^{\top} \bullet V_2 = A_1^{\top} \bullet V_2.
\end{align}

In the [chapter on backpropagation](https://nbviewer.jupyter.org/github/niknow/machine-learning-examples/blob/master/newton_gradient_backprop/backpropagation.ipynb), we also computed the derivative $\nabla(C\circ F)$. There, we defined the functions $G_l := C \circ F_L \circ \ldots \circ F_{l+1} \circ \sigma_l$ and have shown that their gradients $\varepsilon_l := \nabla G_l(z_l)$ satisfy 
\begin{align*}
   \varepsilon_L &= \nabla \sigma_L(z_L)^{\top} \bullet \nabla C(a_L)^{\top}, \\
   \varepsilon_{l} &=  \nabla \sigma_{l}(z_l)^{\top} \bullet A_{l+1}^{\top} \bullet \varepsilon_{l+1}.
\end{align*}
Thus, 
\begin{align*}
    \forall 2 \leq l \leq L: \varepsilon_l = V_l,
\end{align*}
i.e. the backpropagation lemma is just a special case of the adjoint method.